[![Abrir en Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/uxue-sudupe/API-Eustat/blob/main/code_examples/es/tutorial_Python_es.ipynb)

# Tutorial de la API de Eustat con Python

El [banco de datos de Eustat](https://es.eustat.eus/banku/indexArbol.html) contiene una gran cantidad de datos sobre la sociedad, la economía y la población de la Comunidad Autónoma de Euskadi.

Los datos son públicos y accesibles por distintos medios, entre ellos la API, [documentada aquí](https://github.com/uxue-sudupe/API-Eustat). La API facilita enormemente el acceso y la reutilización de los datos. Sin embargo, a menos que se tenga experiencia previa en el acceso a API, su uso puede resultar complejo para un investigador, analista o estudiante.

Este tutorial ofrece una guía rápida sobre cómo acceder a los datos de Eustat a través de su API y descargar su contenido en formato JSON (JavaScript Object Notation) u otro formato permitido. Se muestran ejemplos de cómo descargar los datos y su posterior procesado para obtener tablas en formato dataframe.

Es necesario utilizar algunos paquetes muy extendidos como *pandas* o *requests*, y el paquete *pyjstat*, específico para tratar con Python datos en formato *json-stat*.

In [1]:
!pip install -q pyjstat
import json
import requests
import pandas as pd
from pyjstat import pyjstat

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.0/798.0 kB 8.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


# 1. Obtener el Catálogo de datos Eustat (listado de tablas)

Es posible acceder al listado completo de los conjuntos de datos publicados por Eustat, con detalle de código identificador, título y fecha de última actualización.

Las tablas son las que se encuentran en el banco de datos de Eustat: https://www.eustat.eus/banku/indexArbol.html

Una llamada a la API se compone de una dirección web principal, una llamada a una función y un conjunto de parámetros. La dirección web principal del banco de datos es `https://www.eustat.eus/bankupx/api/v1/[lang]/DB`. En este caso, la llamada a la API se hace con la función **GET**. El único parámetro necesario para obtener el catálogo, es el idioma (lang). La respuesta se recibe en formato JSON.

In [2]:
# URL del punto de acceso (endpoint) del catálogo de datos de Eustat

url = "https://www.eustat.eus/bankupx/api/v1/es/DB"
respuesta = requests.get(url)
respuesta_json = respuesta.json()

print(json.dumps(respuesta_json, indent=2, ensure_ascii=False)[:500])   # Solo para observar el aspecto de los primeros 500 caracteres del JSON formateado y con acentos visibles, no es un paso necesario

[
  {
    "id": "PX__fe_inem06.px",
    "type": "t",
    "text": "Paro registrado de la C.A. de Euskadi por ámbitos territoriales y sexo. 1997 - 2022",
    "updated": "2023-01-09T15:04:41"
  },
  {
    "id": "PX__feinem_inem06.px",
    "type": "t",
    "text": "Paro registrado de la C.A. de Euskadi por ámbitos territoriales y sexo",
    "updated": "2021-02-17T10:04:22"
  },
  {
    "id": "PX__fepycl_pc02cm.px",
    "type": "t",
    "text": "Índice general de precios al consumo (IPC) por Territor


In [3]:
catálogo = pd.DataFrame(respuesta_json)    # convertimos la respuesta a dataframe
catálogo.head()

,id,type,text,updated
0,PX__fe_inem06.px,t,Paro registrado de la C.A. de Euskadi por ámbi...,2023-01-09T15:04:41
1,PX__feinem_inem06.px,t,Paro registrado de la C.A. de Euskadi por ámbi...,2021-02-17T10:04:22
2,PX__fepycl_pc02cm.px,t,Índice general de precios al consumo (IPC) por...,2021-02-16T11:15:00
3,PX_0_fe_pc02cm.px,t,Índice general de precios al consumo (IPC) por...,2022-02-15T10:31:14
4,PX_0_fe_pc02m.px,t,Índice general de precios al consumo (IPC) de ...,2022-02-15T10:31:14


## Idiomas

Se puede pedir el mismo listado en euskera o en inglés. Solo hay que modificar la dirección de la url en la parte que contiene el idioma, `www.eustat.eus/bankupx/api/v1/[lang]/DB`, y reemplazar ***es*** por ***eu*** o ***en***. Por ejemplo, en euskera sería así:

In [4]:
url = "https://www.eustat.eus/bankupx/api/v1/eu/DB"
respuesta = requests.get(url)
respuesta_json = respuesta.json()

katalogoa = pd.DataFrame(respuesta_json)     # convertimos la respuesta a dataframe
katalogoa.head()

,id,type,text,updated
0,PX__fe_inem06.px,t,"Euskal AEko erregistratutako langabezia, lurra...",2023-01-09T15:04:41
1,PX__feinem_inem06.px,t,"Euskal AEko erregistratutako langabezia, lurra...",2021-02-17T10:04:22
2,PX__fepycl_pc02cm.px,t,Euskal AEko eta Estatuko kontsumorako prezioen...,2021-02-16T11:15:00
3,PX_0_fe_pc02cm.px,t,Euskal AEko eta Estatuko kontsumorako prezioen...,2022-02-15T10:31:14
4,PX_0_fe_pc02m.px,t,Euskal AEko eta Estatuko kontsumorako prezioen...,2022-02-15T10:31:14


# 2. Obtener los metadatos de una tabla
Los metadatos de una tabla contienen información sobre el título de la tabla, cuáles son las dimensiones/variables de desagregación que aparecen en la tabla y los valores que toman. Los metadatos se guardan como código o como texto descriptivo.

Para acceder a los metadatos de una tabla concreta, debemos añadirle a la url del banco de datos el código identificador "id" de la tabla de interés. La dirección tendrá este aspecto:
 `www.eustat.eus/bankupx/api/v1/[lang]/DB/[id]`

Accedemos, por ejemplo, a los metadatos de la tabla que contiene los datos de *Población estimada de la C.A. de Euskadi por territorio histórico, sexo y año de nacimiento*, que se encuentra en esta dirección:

https://www.eustat.eus/bankupx/api/v1/es/DB/PX_010154_cepv1_ep17.px

In [5]:
url = "https://www.eustat.eus/bankupx/api/v1/es/DB/PX_010154_cepv1_ep17.px"
respuesta = requests.get(url)
respuesta_json = respuesta.json()

print(json.dumps(respuesta_json, indent=2, ensure_ascii=False)[:500])   # Solo para observar el aspecto de los primeros 500 caracteres del JSON formateado y con acentos visibles, no es un paso necesario

{
  "title": "Población estimada de la C.A. de Euskadi por territorio histórico, sexo y año de nacimiento. 1976/01/01 - 2024/01/01",
  "variables": [
    {
      "code": "territorio histórico",
      "text": "territorio histórico",
      "values": [
        "00",
        "01",
        "48",
        "20"
      ],
      "valueTexts": [
        "C.A. de Euskadi",
        "Araba/Álava",
        "Bizkaia",
        "Gipuzkoa"
      ]
    },
    {
      "code": "sexo",
      "text": "sexo",
      "valu


In [6]:
# Extraemos únicamente el contenido de la clave 'variables', que es lo que nos interesa, y lo convertimos en un DataFrame para facilitar su exploración visual.

variables = respuesta_json.get('variables', [])
variables = pd.DataFrame(variables)
variables

,code,text,values,valueTexts,time
0,territorio histórico,territorio histórico,"[00, 01, 48, 20]","[C.A. de Euskadi, Araba/Álava, Bizkaia, Gipuzkoa]",NaN
1,sexo,sexo,"[10, 20, 30]","[Total, Hombres, Mujeres]",NaN
2,año de nacimiento,año de nacimiento,"[1000, 1109, 1110, 1120, 5800, 5810, 5820, 583...","[Total, 2023, 2022, 2021, 2020, 2019, 2018, 20...",NaN
3,periodo,periodo,"[19760101, 19770101, 19780101, 19790101, 19800...","[1976/01/01, 1977/01/01, 1978/01/01, 1979/01/0...",True


El DataFrame resultante contiene las variables de la tabla, pero sus valores o categorías están agrupados dentro de cada fila como una subestructura.

Puede resultar útil desanidar esta información y generar una tabla en la que se muestren las variables junto a sus valores, incluyendo la correspondencia entre códigos y etiquetas descriptivas.

In [7]:
# Creamos una lista para almacenar las filas
filas = []

# Recorremos cada variable en el DataFrame original
for _, fila in variables.iterrows():
    nombre_variable = fila['code']
    categorias = fila.get('values', [])
    etiquetas = fila.get('valueTexts', [])

    # Emparejamos cada código con su etiqueta
    for codigo, etiqueta in zip(categorias, etiquetas):
        filas.append({
            'variable': nombre_variable,
            'codigo': codigo,
            'etiqueta': etiqueta
        })

# Lo convertimos a DataFrame
tabla_valores = pd.DataFrame(filas)
tabla_valores

,variable,codigo,etiqueta
0,territorio histórico,00,C.A. de Euskadi
1,territorio histórico,01,Araba/Álava
2,territorio histórico,48,Bizkaia
3,territorio histórico,20,Gipuzkoa
4,sexo,10,Total
...,...,...,...
203,periodo,20220101,2022/01/01
204,periodo,20220701,2022/07/01
205,periodo,20230101,2023/01/01
206,periodo,20230701,2023/07/01


# 3. Obtener los datos de una tabla


## Hacer una consulta a una tabla para obtener todos los datos

Para disponer de los datos de las tablas, debemos realizar una consulta a la misma dirección del banco de datos con la función **POST**. Utilizamos el módulo *pyjstat* para trabajar con los datos que vamos a descargar en formato *json-stat*. En este ejemplo, consultamos todos los datos de la tabla del *Producto interior bruto (PIB) de la C.A. de Euskadi (oferta) por territorio histórico, rama de actividad (A-38), tipo de dato y de medida*.

In [8]:
# URL del punto de acceso (endpoint) de la tabla
url = "https://www.eustat.eus/bankupx/api/v1/es/DB/PX_170112_cpib_pib01d.px"

# Cuerpo de la petición: sin filtros (toda la tabla). Por defecto, el resultado se obtiene en formato JSON-stat v.1.2
query = { "query": []}


sesion = requests.Session()
respuesta = sesion.post(url, json=query)                    #  Se envía una solicitud POST a la URL de la tabla (consulta API)
respuesta_json = json.loads(respuesta.content.decode("utf-8-sig"))     #   Convierte la respuesta en texto legible

resultado = pyjstat.from_json_stat(respuesta_json, naming="label", value="value") # Convertimos el JSON-stat a DataFrame con etiquetas descriptivas (label)
PIB = resultado[0]                    # Nos interesa solo la primera lista de DataFrames
PIB.head()

,territorio histórico,sector,tipo de dato,tipo de medida,periodo,value
0,C.A. de Euskadi,"1. Agricultura, ganadería y pesca",Nivel,Precios corrientes (miles euros),1995,449513.0
1,C.A. de Euskadi,"1. Agricultura, ganadería y pesca",Nivel,Precios corrientes (miles euros),1996,439283.0
2,C.A. de Euskadi,"1. Agricultura, ganadería y pesca",Nivel,Precios corrientes (miles euros),1997,466735.0
3,C.A. de Euskadi,"1. Agricultura, ganadería y pesca",Nivel,Precios corrientes (miles euros),1998,564307.0
4,C.A. de Euskadi,"1. Agricultura, ganadería y pesca",Nivel,Precios corrientes (miles euros),1999,605399.0


## Obtener los datos de una tabla, con códigos

Es posible que nos interese disponer de estos datos con los códigos de las variables, y no con las etiquetas descriptivas o literales. En ese caso, solicitamos que los nombres se capturen como código (naming="id") y no como texto (naming="label").

In [9]:
resultado = pyjstat.from_json_stat(respuesta_json, naming='id', value='value')
PIB_codigo = resultado[0]              # Nos interesa solo la primera lista de DataFrames
PIB_codigo

,territorio histórico,sector,tipo de dato,tipo de medida,periodo,value
0,00,01,10,10,1995,449513.0
1,00,01,10,10,1996,439283.0
2,00,01,10,10,1997,466735.0
3,00,01,10,10,1998,564307.0
4,00,01,10,10,1999,605399.0
...,...,...,...,...,...,...
17915,20,70,20,20,2018,3.7
17916,20,70,20,20,2019,2.6
17917,20,70,20,20,2020,-9.9
17918,20,70,20,20,2021,5.9


## Obtener los datos de una tabla, con una consulta de datos

Solicitar demasiados datos para luego tener que desechar la mitad es desperdiciar espacio y recursos. Por ello, cuando nos interesa disponer de los datos de una selección de valores, y no todo el contenido de las tablas, debemos realizar una consulta seleccionando las variables y valores de interés.

Para definir la consulta, es necesario conocer el nombre/código de las variables y los valores, que es la información que hemos recibido en el punto anterior, **2. Obtener los metadatos de una tabla**.

Pero para facilitar este paso, la propia web de Eustat dispone de una interfaz de selección de variables y valores para cada tabla, donde podremos seleccionar los datos que nos interesan fácilmente, y obtener la consulta en formato JSON.

Por ejemplo, si queremos datos de la tasa de paro por sexo, iríamos a la página en la que se seleccionan las variables para esta tabla del banco de datos:

https://www.eustat.eus/bankupx/pxweb/es/DB/-/PX_050403_cpra_tab01.px

Y hacemos la selección que nos interesa: seleccionamos los datos de la tasa de paro para la C.A. de Euskadi, los 2 sexos (y el total) y los datos trimestrales de los 3 últimos años.

#### Ejemplo de Selección:
![Paro Selección](https://raw.githubusercontent.com/uxue-sudupe/API-Eustat/main/img/Paro_seleccion.png)


Tras hacer la selección pulsamos "Continuar" y "Disponer de esta tabla en su aplicación". En este apartado aparecerán tanto la url a la que hay que hacer la petición como la consulta json, que será la que incluiremos en nuestro código.


#### Ejemplo de Consulta:
![Paro Consulta](https://raw.githubusercontent.com/uxue-sudupe/API-Eustat/main/img/Paro_consulta.png)


In [10]:
url = "https://www.eustat.eus/bankupx/api/v1/es/DB/PX_050403_cpra_tab01.px"
query = {"query": [{"code": "tasa (%)","selection": {"filter": "item", "values": ["30"] }},
        {"code": "territorio histórico","selection": {"filter": "item", "values": [ "00"]}},
        {"code": "trimestre","selection": {"filter": "item","values": ["20", "30", "40","50" ]}},
        {"code": "periodo","selection": {"filter": "item","values": ["2023", "2024","2025" ]}}],
         "response": {"format": "json-stat"}}


sesion = requests.Session()
respuesta = sesion.post(url, json=query)                 #  Se envía una solicitud POST a la URL de la tabla (consulta API)
respuesta_json = json.loads(respuesta.content.decode('utf-8-sig'))     #   Convierte la respuesta en texto legible
resultados = pyjstat.from_json_stat(respuesta_json, naming='label', value='value')  #  Convierte la respuesta JSON-stat (formato especial de datos estadísticos) en un DataFrame de pandas.
paro= resultados[0]        # Nos interesa solo la primera lista de DataFrames
paro

,tasa (%),territorio histórico,sexo,trimestre,periodo,value
0,Tasa de paro,C.A. de Euskadi,Total,Trimestre 1,2023,8.9
1,Tasa de paro,C.A. de Euskadi,Total,Trimestre 1,2024,7.9
2,Tasa de paro,C.A. de Euskadi,Total,Trimestre 1,2025,7.0
3,Tasa de paro,C.A. de Euskadi,Total,Trimestre 2,2023,7.6
4,Tasa de paro,C.A. de Euskadi,Total,Trimestre 2,2024,7.5
5,Tasa de paro,C.A. de Euskadi,Total,Trimestre 2,2025,NaN
6,Tasa de paro,C.A. de Euskadi,Total,Trimestre 3,2023,7.7
7,Tasa de paro,C.A. de Euskadi,Total,Trimestre 3,2024,7.2
8,Tasa de paro,C.A. de Euskadi,Total,Trimestre 3,2025,NaN
9,Tasa de paro,C.A. de Euskadi,Total,Trimestre 4,2023,7.1


### Modificaciones a la consulta

Cuando se realiza "filter": "item" sobre una variable, se seleccionan los valores que se quieren filtrar (uno o varios).

Si se seleccionan todos los valores, no se está filtrando sobre esa variable, y no aparecerá en la consulta. Tiene el mismo efecto que eliminar la linea que empieza por `{"code":"variable"....}` como haremos ahora con la línea de código de la variable "territorio histórico".

También podemos usar el filtro "top", para seleccionar unicamente un número determinado con los últimos valores de la variable. Este filtro "top" se utiliza principalmente con la variable "periodo", "año" o similares.

In [11]:
url = "https://www.eustat.eus/bankupx/api/v1/es/DB/PX_050403_cpra_tab01.px"
query = {"query": [{"code": "tasa (%)","selection": {"filter": "item", "values": ["30"] }},
    #   {"code": "territorio histórico","selection": {"filter": "*", "values": [ "*"]}},    si eliminamos esta línea, no filtramos la variable "territorio histórico"
        {"code": "trimestre","selection": {"filter": "item","values": ["20", "30", "40","50" ]}},
        {"code": "periodo","selection": {"filter": "top","values": ["2"]}}],             #  filtro "top", últimos 2 valores de la variable "periodo"
         "response": {"format": "json-stat"}}

sesion = requests.Session()
respuesta = sesion.post(url, json=query)
respuesta_json = json.loads(respuesta.content.decode('utf-8-sig'))
resultados = pyjstat.from_json_stat(respuesta_json, naming='label', value='value')
paro= resultados[0]
paro

,tasa (%),territorio histórico,sexo,trimestre,periodo,value
0,Tasa de paro,C.A. de Euskadi,Total,Trimestre 1,2024,7.9
1,Tasa de paro,C.A. de Euskadi,Total,Trimestre 1,2025,7.0
2,Tasa de paro,C.A. de Euskadi,Total,Trimestre 2,2024,7.5
3,Tasa de paro,C.A. de Euskadi,Total,Trimestre 2,2025,NaN
4,Tasa de paro,C.A. de Euskadi,Total,Trimestre 3,2024,7.2
...,...,...,...,...,...,...
91,Tasa de paro,Gipuzkoa,Mujeres,Trimestre 2,2025,NaN
92,Tasa de paro,Gipuzkoa,Mujeres,Trimestre 3,2024,5.2
93,Tasa de paro,Gipuzkoa,Mujeres,Trimestre 3,2025,NaN
94,Tasa de paro,Gipuzkoa,Mujeres,Trimestre 4,2024,6.0


## Otros formatos de salida

La salida por defecto de las peticiones API a los datos de las tablas es JSON-stat (versión 1.2). El formato JSON es el más común en APIs, permite un tratamiento flexible de los datos, adecuado para procesamiento automático en scripts (R, Python, etc.), y
evita ambigüedades que pueden presentarse en archivos planos como CSV.

También es posible descargar los datos pedidos por API en formatos como csv y excel. Estas salidas pueden ser más adecuadas con usuarios finales no técnicos, en procesos en los que se necesitan csv o excel como entrada (creación de dashboards desde csv/excel) o cuando el objetivo no es el análisis sino facilitar el acceso al público de la datos descargados por API.

A continuación se muestran las salidas csv (3 tipos) y excel, y se guarda una copia de cada una.

In [12]:
# Salida en CSV - texto plano

url ="https://www.eustat.eus/bankupx/api/v1/es/DB/PX_010901_cecv_ni11_1.px"
query = {"query": [],
  "response": {"format": "csv"}}


respuesta_csv = requests.post(url, json=query)

# Para ver el contenido de la respuesta. Este paso no es necesario
texto_csv = respuesta_csv.content.decode("windows-1252")
print(texto_csv[:1000])   # Aspecto de los primeros 1000 caracteres del JSON formateado y con acentos visibles

# Guardar el archivo CSV localmente
with open("lengua_materna.csv", "wb") as f:
    f.write(respuesta_csv.content)

"lengua materna","zona","1989","1994","1999","2004","2009","2014","2019","2024"
"Total","C.A. de Euskadi",1997.2,1995.5,1991.0,1993.0,2051.6,2023.6,2079.0,2120.7
"Total","Llanada Alavesa con Cantabria Alavesa",236.5,237.6,246.3,254.5,272.2,270.0,279.4,289.6
"Total","Resto de Álava",22.1,21.1,22.7,24.5,26.8,28.4,27.6,28.8
"Total","Gran Bilbao",871.6,862.7,847.8,839.2,842.5,804.4,813.0,833.1
"Total","Duranguesado",84.3,86.1,85.8,78.9,87.1,94.3,95.0,94.3
"Total","Gernika-Bermeo con Plentzia-Mungia y Markina-Ondarroa",99.5,101.6,103.6,111.8,116.8,115.3,125.3,129.8
"Total","Arratia-Nervión con Encartaciones",47.5,48.1,47.8,49.3,50.2,53.3,53.5,54.0
"Total","Donostia-San Sebastian con Bajo Bidasoa",353.5,357.5,361.9,364.6,374.6,375.6,389.6,391.5
"Total","Alto Deba con Bajo Deba y Urola Costa",174.7,173.8,172.4,171.0,176.4,174.2,182.7,186.2
"Total","Tolosa con Goierri",107.5,106.8,102.8,99.3,105.0,108.1,112.9,113.2
"Euskera","C.A. de Euskadi",399.3,369.6,388.4,359.0,370.1,367.5,365.


In [13]:
# Salida en CSV2 - formato "pivot-friendly", compatible con tablas dinámicas

url ="https://www.eustat.eus/bankupx/api/v1/es/DB/PX_010901_cecv_ni11_1.px"
query = {"query": [],
  "response": {"format": "csv2"}}


respuesta_csv2 = requests.post(url, json=query)

# Guardar el archivo CSV localmente
with open("lengua_materna_csv2.csv", "wb") as f:
    f.write(respuesta_csv2.content)

In [14]:
# Salida en CSV3 - igual que CSV2, pero con códigos en lugar de texto

url ="https://www.eustat.eus/bankupx/api/v1/es/DB/PX_010901_cecv_ni11_1.px"
query = {"query": [],
  "response": {"format": "csv3"}}


respuesta_csv3 = requests.post(url, json=query)

# Guardar el archivo CSV localmente
with open("lengua_materna_csv3.csv", "wb") as f:
    f.write(respuesta_csv3.content)


In [15]:
# Salida en Excel

url ="https://www.eustat.eus/bankupx/api/v1/es/DB/PX_010901_cecv_ni11_1.px"
query = {"query": [],
  "response": {"format": "xlsx"}}

respuesta = requests.post(url, json=query)

# Guardar el archivo Excel localmente
with open("lengua_materna.xlsx", "wb") as f:
    f.write(respuesta.content)